In [ ]:
#Grok 3 refined

import os
import base64
from pathlib import Path
from PIL import Image
from IPython.display import display, HTML, Image as IPImage
import ipywidgets as widgets
import platform
from metadata_utils import extract_metadata  # Import from separate module

def open_file(file_path: str) -> None:
    """Open a file or folder in the default application."""
    try:
        file_path = str(Path(file_path))  # Normalize path
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Path does not exist: {file_path}")
        if platform.system() == "Windows":
            os.startfile(file_path)
        else:
            import subprocess
            subprocess.run(["open", file_path], check=True)  # macOS
            # For Linux: subprocess.run(["xdg-open", file_path], check=True)
        print(f"Success: Opened {file_path}")
    except Exception as e:
        print(f"Error opening {file_path}: {e}")

def display_image(file_path: str, scale: float = 0.3) -> None:
    """Display an image at a scaled size in the notebook."""
    try:
        with Image.open(file_path) as img:
            width, height = img.size
            display_width = int(width * scale)
            display_height = int(height * scale)
            display(IPImage(filename=file_path, width=display_width, height=display_height))
    except Exception as e:
        print(f"Error displaying image {file_path}: {e}")

def display_html(file_path: str, filename: str) -> None:
    """Display filename, path, and a link to toggle an inline image."""
    try:
        # Encode image as base64 for inline display
        with open(file_path, "rb") as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        html = f"""
        <div>
            <p><strong>Filename:</strong> {filename}</p>
            <p><strong>File Path:</strong> {file_path}</p>
            <p><a href="data:image/png;base64,{image_data}" target="_blank">Click to open image in new tab</a></p>
        </div>
        """
        display(HTML(html))
    except Exception as e:
        print(f"Error displaying HTML for {file_path}: {e}")

def display_buttons(file_path: str) -> None:
    """Display buttons to open the image file and its parent folder."""
    button_file = widgets.Button(
        description="Open Image", 
        layout={'width': '150px'}, 
        style={'button_color': 'lightblue'}
    )
    button_folder = widgets.Button(
        description="Open Folder", 
        layout={'width': '150px'}, 
        style={'button_color': 'lightgreen'}
    )

    def on_file_button_clicked(b, path=file_path):
        open_file(path)
    def on_folder_button_clicked(b, path=str(Path(file_path).parent)):
        open_file(path)

    button_file.on_click(on_file_button_clicked)
    button_folder.on_click(on_folder_button_clicked)

    display(widgets.HBox([button_file, button_folder]))

def display_img_by_lora(
    input_dirs: list[str],
    lora_name: str,
    formats: list[str] = ['.png']
) -> None:
    """Display images filtered by Lora name."""

    for input_dir in input_dirs:
        input_path = Path(input_dir)
        if not input_path.exists():
            print(f"Skipping non-existent directory: {input_dir}")
            continue

        for file_path in input_path.iterdir():
            if file_path.suffix.lower() in formats:
                try:
                    metadata = extract_metadata(file_path)
                    if not metadata:
                        continue

                    lora = metadata.get('Lora hashes', '').strip('"').split(':')[0]
                    if lora and lora == lora_name:
                        # Display image and controls
                        display_image(str(file_path))
                        # display_html(str(file_path), file_path.name) #Currently not work in Jupyter notebook
                        display_buttons(str(file_path))

                except Exception as e:
                    print(f"Error processing {file_path.name}: {e}")

#--------------------------------------- Configuration
INPUT_DIRS = [
    r'S:\webui_forge_cu121_torch231\webui\outputs\txt2img-images\2025-05-23',
    r'S:\webui_forge_cu121_torch231\webui\outputs\txt2img-images\2025-05-22',
    r'S:\webui_forge_cu121_torch231\webui\outputs\txt2img-images\2025-05-21',
    r'S:\Fooocus_win64_2-5-0\Fooocus_win64_2-5-0\Fooocus\outputs\2025-05-21',
    r'S:\Fooocus_win64_2-5-0\Fooocus_win64_2-5-0\Fooocus\outputs\2025-05-19'
]
LORA_NAME = 'dragonnest_academic_250519a-000008'

#----------------------------------------Run the display function
if __name__ == "__main__":
    display_img_by_lora(INPUT_DIRS, LORA_NAME)